# Experiment 003: Ensemble Approach

Combine the best configuration for each N from multiple sources to create an improved submission.

In [1]:
import numpy as np
import pandas as pd
import glob
import os
from numba import njit
import math

print('Libraries loaded')

Libraries loaded


In [2]:
# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x,y

@njit
def score_group(xs, ys, degs, tx, ty):
    """Calculate score for a single N configuration."""
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

def strip(a):
    """Remove 's' prefix from values."""
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

tx, ty = make_polygon_template()
print('Scoring functions defined')

Scoring functions defined


In [3]:
# List all available submission files
sources = [
    '/home/code/preoptimized_submission.csv',
    '/home/code/datasets/santa-2025.csv',
    '/home/code/datasets/71.97.csv',
    '/home/code/datasets/72.49.csv',
    '/home/code/datasets/submission.csv',  # telegram
    '/home/code/datasets/jazivxt_output/submission.csv',
    '/home/code/datasets/eazy_output/submission.csv',
    '/home/code/datasets/ashraful_output/submission.csv',
]

# Check which files exist
existing_sources = []
for src in sources:
    if os.path.exists(src):
        existing_sources.append(src)
        print(f'Found: {src}')
    else:
        print(f'Missing: {src}')

print(f'\nTotal sources: {len(existing_sources)}')

Found: /home/code/preoptimized_submission.csv
Found: /home/code/datasets/santa-2025.csv
Found: /home/code/datasets/71.97.csv
Found: /home/code/datasets/72.49.csv
Found: /home/code/datasets/submission.csv
Found: /home/code/datasets/jazivxt_output/submission.csv
Found: /home/code/datasets/eazy_output/submission.csv
Found: /home/code/datasets/ashraful_output/submission.csv

Total sources: 8


In [4]:
# Load all sources and compute scores per N
best = {n: {'score': 1e300, 'data': None, 'src': None} for n in range(1, 201)}

for fp in existing_sources:
    try:
        df = pd.read_csv(fp)
    except Exception as e:
        print(f'Error loading {fp}: {e}')
        continue
    
    if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
        print(f'Invalid columns in {fp}')
        continue
    
    df = df.copy()
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    
    for n, g in df.groupby('N'):
        if n < 1 or n > 200:
            continue
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        sc = score_group(xs, ys, ds, tx, ty)
        if sc < best[n]['score']:
            best[n]['score'] = float(sc)
            best[n]['data'] = g.drop(columns=['N']).copy()
            best[n]['src'] = os.path.basename(fp)

print('Loaded all sources and computed per-N scores')

Loaded all sources and computed per-N scores


In [5]:
# Override N=1 with optimal configuration (0, 0, 45 degrees)
manual_data = pd.DataFrame({
    'id': ['001_0'],
    'x': ['s0.0'],
    'y': ['s0.0'],
    'deg': ['s45.0']
})
xs = strip(manual_data['x'].to_numpy())
ys = strip(manual_data['y'].to_numpy())
ds = strip(manual_data['deg'].to_numpy())
sc = score_group(xs, ys, ds, tx, ty)

print(f'N=1 optimal score: {sc:.6f}')
print(f'N=1 current best: {best[1]["score"]:.6f}')

if sc < best[1]['score']:
    best[1]['score'] = float(sc)
    best[1]['data'] = manual_data.copy()
    best[1]['src'] = 'optimal_n1'
    print('Replaced N=1 with optimal configuration')
else:
    print('N=1 already optimal')

N=1 optimal score: 0.661250
N=1 current best: 0.661250
N=1 already optimal


In [6]:
# Build ensemble submission
rows = []
used = {}
total = 0.0

for n in range(1, 201):
    entry = best[n]
    if entry['data'] is None:
        print(f'Warning: No data for N={n}')
        continue
    rows.append(entry['data'])
    used[entry['src']] = used.get(entry['src'], 0) + 1
    total += entry['score']

print('\n=== Source Usage ===')
for src, count in sorted(used.items(), key=lambda x: -x[1]):
    print(f'  {src}: {count} N values')

print(f'\n=== Ensemble Score ===')
print(f'Total score: {total:.6f}')
print(f'Target: 68.922808')
print(f'Gap: {total - 68.922808:.6f}')


=== Source Usage ===
  santa-2025.csv: 192 N values
  submission.csv: 6 N values
  preoptimized_submission.csv: 2 N values

=== Ensemble Score ===
Total score: 70.676092
Target: 68.922808
Gap: 1.753284


In [7]:
# Create final submission
out = pd.concat(rows, ignore_index=True)
out['sn'] = out['id'].str.split('_').str[0].astype(int)
out['si'] = out['id'].str.split('_').str[1].astype(int)
out = out.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
out = out[['id', 'x', 'y', 'deg']]

# Save submission
submission_path = '/home/submission/submission.csv'
out.to_csv(submission_path, index=False)
print(f'Saved ensemble submission to {submission_path}')
print(f'Total rows: {len(out)}')

Saved ensemble submission to /home/submission/submission.csv
Total rows: 20100


In [8]:
# Verify the submission
df_verify = pd.read_csv(submission_path)
verify_total = 0.0
for n in range(1, 201):
    g = df_verify[df_verify['id'].str.startswith(f'{n:03d}_')]
    xs = strip(g['x'].to_numpy())
    ys = strip(g['y'].to_numpy())
    ds = strip(g['deg'].to_numpy())
    sc = score_group(xs, ys, ds, tx, ty)
    verify_total += sc

print(f'Verified ensemble score: {verify_total:.6f}')
print(f'Expected: {total:.6f}')
print(f'Match: {abs(verify_total - total) < 1e-6}')

Verified ensemble score: 70.676092
Expected: 70.676092
Match: True


In [9]:
# Show worst N values
print('\n=== Worst N Values (top 20) ===')
scores_list = []
for n in range(1, 201):
    scores_list.append((n, best[n]['score'], best[n]['src']))

scores_list.sort(key=lambda x: -x[1])
for n, score, src in scores_list[:20]:
    print(f'  N={n:3d}: {score:.6f} from {src}')


=== Worst N Values (top 20) ===
  N=  1: 0.661250 from preoptimized_submission.csv
  N=  2: 0.450779 from santa-2025.csv
  N=  3: 0.434745 from preoptimized_submission.csv
  N=  5: 0.416850 from santa-2025.csv
  N=  4: 0.416545 from santa-2025.csv
  N=  7: 0.399897 from santa-2025.csv
  N=  6: 0.399610 from santa-2025.csv
  N=  9: 0.387415 from santa-2025.csv
  N=  8: 0.385407 from santa-2025.csv
  N= 15: 0.379203 from santa-2025.csv
  N= 10: 0.376630 from santa-2025.csv
  N= 21: 0.376451 from santa-2025.csv
  N= 20: 0.376057 from santa-2025.csv
  N= 11: 0.375736 from santa-2025.csv
  N= 22: 0.375258 from santa-2025.csv
  N= 16: 0.374128 from santa-2025.csv
  N= 26: 0.373997 from santa-2025.csv
  N= 12: 0.372724 from santa-2025.csv
  N= 13: 0.372323 from santa-2025.csv
  N= 25: 0.372144 from santa-2025.csv
